In [15]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import f_regression
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
import sys
import pickle 
from collections import Counter

# Initialize linear regression
model = Ridge()

# Set random seed 
random_state = 12

# Set minimum number of cells 
min_cells = 7

# Read Slide-seq transformed coorindates
df = pd.read_csv("data/Aligned_glomeruli.csv", header = 0, names=["OR", "X", "Y", "mouse"])
df.set_index("OR", inplace=True, drop=True)
ORs = sorted(list(df.index))
print("{} ORs with Slide-seq coordinates".format(len(ORs)))

# Convert coordinates to a dictionary keyed on OR
coords = df[["X", "Y"]].to_dict("index")

# Fit scaler so it stores the min and max of the x and y coordinates
scaler_y = MinMaxScaler().fit(df[["X", "Y"]].values)

# Read in PCs
df = pd.read_csv("data/LR_OSN_all_50PCs_3000G.csv", index_col=0)

n_col = len(df.columns)
df.rename(columns={"observed": "OR"}, inplace=True)

print(df.head())

# Only retain PCs for ORs with slide-seq coordinates
df = df[df["OR"].isin(ORs)]

ORs_to_exclude = [x for x in ORs if len(df[df["OR"] == x]) < min_cells]
print(ORs_to_exclude)
df = df[~df["OR"].isin(ORs_to_exclude)]


print("{} ORs with Slide-seq coordinates but no gene expression".format(len(ORs_to_exclude)))
ORs = list(df.OR.unique())
print("{} ORs remaining".format(len(ORs)))

# Z-score normalize PCs
scaler_x = StandardScaler().fit(df.values[:,0:(n_col - 1)])

# Generating train and eval set 
X_train = df.values[:,0:(n_col - 1)]
X_train = scaler_x.transform(X_train)

train_ors = np.array(df.iloc[:,(n_col - 1)])

# Use KNN to oversample less represented cells to remove any potential bias
# created by ORs with large numbers of cells
counter = Counter(train_ors)
oversample = SMOTE(random_state=random_state)
X_train, train_ors = oversample.fit_resample(X_train, train_ors)
counter = Counter(train_ors)
max_cells = np.max(list(counter.values()))


# Generate coordinate vectors from coordinate dictionary and ORs
y_train = []
for i in range(train_ors.shape[0]):
    y_train.append([coords[train_ors[i]]["X"], coords[train_ors[i]]["Y"]])
        

y_train = np.array(y_train)

# Scale coordinates
y_train = scaler_y.transform(y_train)

# Fit Ridge regression
model.fit(X_train, y_train)


# Pickle model
model_file = "data/model.pkl"
pickle.dump(model, open(model_file, 'wb'))

np.savetxt("data/coefficients.txt", np.transpose(model.coef_))

f_x, p_x = f_regression(X_train, y_train[:,0])
f_y, p_y = f_regression(X_train, y_train[:,1])

with open("data/coefficient_p_values.txt", "w") as f:
    for i in range(p_x.shape[0]):
        print("x", i+1, "{:e}".format(p_x[i]), file = f, sep="\t")
        
    for i in range(p_y.shape[0]):
        print("y", i+1, "{:e}".format(p_y[i]), file = f, sep="\t")



# Run lr on bootstrap samples to get prediction intervals
STDS_X = []
STDS_Y = []
print(len(ORs))
for i in range(100):
    bootstrap_ORs = np.random.choice(ORs, len(ORs))
    
    pc_arrays = []
    coord_arrays = []
    for x in bootstrap_ORs:
        indices = [j for j in range(len(train_ors)) if train_ors[j] == x]
        pc_arrays.append(X_train[indices,:])
        coord_arrays.append(y_train[indices,:])
        
    
    model = Ridge()
    model.fit(np.vstack(pc_arrays), np.vstack(coord_arrays))
    
    preds = model.predict(np.vstack(pc_arrays))
    x_preds = []
    y_preds = []
    x_true = []
    y_true = []
    coord_arrays = scaler_y.inverse_transform(np.vstack(coord_arrays))
    
    for j, x in enumerate(bootstrap_ORs):
        x_preds.append(np.mean(preds[j*max_cells:(j*max_cells + max_cells),0]))
        y_preds.append(np.mean(preds[j*max_cells:(j*max_cells + max_cells),1]))
        x_true.append(np.mean(coord_arrays[j*max_cells:(j*max_cells + max_cells),0]))
        y_true.append(np.mean(coord_arrays[j*max_cells:(j*max_cells + max_cells),1]))
        
        
    x_preds = np.array(x_preds)
    y_preds = np.array(y_preds)
    
    
    mean_preds = np.transpose(np.vstack((x_preds, y_preds)))
    mean_preds = scaler_y.inverse_transform(mean_preds)
    x_true = np.array(x_true)
    y_true = np.array(y_true)
    
    
    stdev_x = np.sqrt(1/(len(ORs)-2) * np.sum((x_true - mean_preds[:,0])**2))
    stdev_y = np.sqrt(1/(len(ORs)-2) * np.sum((y_true - mean_preds[:,1])**2))
    STDS_X.append(stdev_x)
    STDS_Y.append(stdev_y)
    
print(np.mean(STDS_X), np.mean(STDS_Y))

# Make predictions on all other ORs
df = pd.read_csv("data/LR_OSN_all_50PCs_3000G.csv", index_col=0)
df.rename(columns={"observed": "OR"}, inplace=True)
n_col = len(df.columns)

all_ORs = list(df.OR.unique())
ORs_of_interest = [x for x in all_ORs if len(df[df["OR"] == x]) >= min_cells]
print(len(ORs_of_interest))

with open("data/map_654.txt", "w") as f:
    for OR in ORs_of_interest:
        X = df[df["OR"] == OR].values[:,0:(n_col - 1)]
        n = X.shape[0]
        X = scaler_x.transform(X)
        pred = scaler_y.inverse_transform(model.predict(X))
        pred = np.mean(pred, axis=0)
        x_pred, y_pred = pred
        print(OR,n,"predicted",x_pred,y_pred, file=f, sep="\t")
        if OR in ORs:
            print(OR, n, "slide-seq", coords[OR]["X"], coords[OR]["Y"], file=f, sep="\t")

79 ORs with Slide-seq coordinates
                       harmony_1  harmony_2  harmony_3  harmony_4  harmony_5  \
V3F1_CAACGGCAGTAAACAC   2.558849   0.506802  -2.369886   1.874797   1.657120   
V3F3_GAGACCCTCTGAGCAT  -0.964222  -6.844951  -1.793788  -0.045801  -0.452054   
V3F1_GAGGGATGTAGCCCTG  10.467781   2.561541   2.176045   1.047865  -2.936098   
V3F3_ACGTAGTCAGGACAGT   3.855414  -3.486225   2.411903  -5.956743  -2.668481   
V3M1_ACCAAACCAAGTGACG   3.137225   3.418350   4.027432  -0.138194   0.723199   

                       harmony_6  harmony_7  harmony_8  harmony_9  harmony_10  \
V3F1_CAACGGCAGTAAACAC  -0.746696   0.330887   1.457195  -0.677314   -0.956127   
V3F3_GAGACCCTCTGAGCAT  -0.311725  -0.178832  -0.569134   0.065686    0.931873   
V3F1_GAGGGATGTAGCCCTG  -1.162711   0.215706   0.705806   0.393240    0.137744   
V3F3_ACGTAGTCAGGACAGT   2.818753  -3.793611   2.078421  -2.440817   -1.163426   
V3M1_ACCAAACCAAGTGACG   4.358617  -0.906037  -0.747231  -2.364545    4.069403   